In [1]:
import BiomationScripter as BMS
import BiomationScripter.OTProto as OTP

In [2]:
def run(protocol):
    
    ########################
    # General Information: #
    ########################
    
    Custom_Labware_Directory = "../Custom_Labware/"
    
    Calibrants_Labware_Type = "3dprinted_24_tuberack_1500ul"
    
    Calibrants = [
        "fluorescein", # 10 uM, 1 mL
        "cascade blue", # 10 uM, 1 mL
        "sulforhodamine 101", # 2 uM, 1 mL
        "silica nanoparticles 950 nm" # 3e9 spheres/mL in ddH2O, 1 mL
    ]
    
    Calibrants_Source_Positions = [
        "A1",
        "A2",
        "A3",
        "A4",
    ]
    
    Buffer_Labware_Type = "3dprinted_15_tuberack_15000ul"
    
    Buffers = [
        "water", # 6 mL
        "PBS" # 6 mL
    ]
    
    Buffer_Source_Positions = [
        "A1",
        "A2"
    ]
    
    p300_starting_tip = "A1"
    
    #####################################################################################
    Calibration_Plate_Type = "nunclondeltasurface163320_96_wellplate_250ul" # TEMPORARY #
    #####################################################################################
    
    ############################
    # Set up pipettes and tips #
    ############################
    
    Deck_Slot = OTP.next_empty_slot(protocol)
    tip_rack_300uL = OTP.load_labware(
        protocol,
        "opentrons_96_tiprack_300ul",
        Deck_Slot
    )
    
    p300 = protocol.load_instrument(
        "p300_single_gen2",
        "right",
        tip_racks = [tip_rack_300uL]
    )
    
    p300.starting_tip = tip_rack_300uL.well(p300_starting_tip)

    ##########################
    # Set up source labware: #
    ##########################
    
    Deck_Slot = OTP.next_empty_slot(protocol)
    Calibrants_Labware = OTP.load_labware(
        protocol,
        Calibrants_Labware_Type,
        Deck_Slot,
        Custom_Labware_Directory)
    
    Deck_Slot = OTP.next_empty_slot(protocol)
    Buffer_Labware = OTP.load_labware(
        protocol,
        Buffer_Labware_Type,
        Deck_Slot,
        Custom_Labware_Directory)
    
    #############################
    # Set up destination plate: #
    #############################
    
    Deck_Slot = OTP.next_empty_slot(protocol)
    Calibration_Plate = OTP.load_labware(
        protocol,
        Calibration_Plate_Type,
        Deck_Slot,
        Custom_Labware_Directory)
    
    Calibration_Plate_Layout = BMS.PlateLayout("Calibration Plate", Calibration_Plate)
    Calibration_Plate_Layout.define_format(
        Rows = 8,
        Columns = 12
    )
    
    #######################
    # Set up liquid class #
    #######################
    
    Liquids = BMS.Liquids()
    
    for calibrant, well in zip(Calibrants, Calibrants_Source_Positions):
        Liquids.add_liquid(calibrant, Calibrants_Labware, well)
        
    for buffer, well in zip(Buffers, Buffer_Source_Positions):
        Liquids.add_liquid(buffer, Buffer_Labware, well)
        
    #####################
    # Set up waste tube #
    #####################
    
    Waste_Tube = Buffer_Labware.wells_by_name()["A3"]

    ##########################
    #  Start liquid handling #
    ##########################
    
    # Add PBS to wells
    Destination_Well_Range = Calibration_Plate_Layout.get_well_range(Well_Range = "A2:D12")
    PBS_Labware = Liquids.get_liquid_labware("PBS")
    PBS_Source_Well = Liquids.get_liquid_well("PBS")
    PBS_Source = PBS_Labware.wells_by_name()[PBS_Source_Well]
    
    p300.pick_up_tip()
    for destination_well in Destination_Well_Range:
        Destination = Calibration_Plate.wells_by_name()[destination_well]
        p300.transfer(100, PBS_Source, Destination, new_tip = "never")
    p300.drop_tip()
    
    
    # Add Water to wells
    Destination_Well_Range = Calibration_Plate_Layout.get_well_range(Well_Range = "E2:H12")
    water_Labware = Liquids.get_liquid_labware("water")
    water_Source_Well = Liquids.get_liquid_well("water")
    water_Source = water_Labware.wells_by_name()[water_Source_Well]
    
    p300.pick_up_tip()
    for destination_well in Destination_Well_Range:
        Destination = Calibration_Plate.wells_by_name()[destination_well]
        p300.transfer(100, water_Source, Destination, new_tip = "never")
    p300.drop_tip()
    
    # Serial Dilutions
    
    Serial_Dilutions = [
        ["fluorescein", ["A","B"]],
        ["sulforhodamine 101", ["C","D"]],
        ["cascade blue", ["E","F"]],
        ["silica nanoparticles 950 nm", ["G","H"]],
    ]
    
    column_names = range(1,11)
    
    for serial_dilution in Serial_Dilutions:
        calibrant, row_names = serial_dilution
        
        Source_Labware = Liquids.get_liquid_labware(calibrant)
        Source_Well = Liquids.get_liquid_well(calibrant)
        Source = Source_Labware.wells_by_name()[Source_Well]
        
        for row in row_names:
            p300.pick_up_tip()
            
            ## Add calibrant to first well
            Destination = Calibration_Plate.wells_by_name()["{}1".format(row)]
        
            p300.transfer(200, Source, Destination, new_tip = "never")
            
            ## Prepare serial dilutions
            for column in column_names:
                source_well = "{}{}".format(row, column)
                source = Calibration_Plate.wells_by_name()[source_well]
                destination_well = "{}{}".format(row, column+1)
                destination = Calibration_Plate.wells_by_name()[destination_well]
            
                p300.transfer(100, source, destination, mix_after = (3, 100), new_tip = "never")
                
            # Dispose extra 100 from column 11
            p300.transfer(100, destination, Waste_Tube, new_tip = "never")
            
            p300.drop_tip()

In [3]:
#####################################################################
# Use this cell if simulating the protocol, otherwise comment it out #
#####################################################################

#########################################################################################################
# IMPORTANT - the protocol will not upload to the opentrons if this cell is not commented out or removed #
#########################################################################################################

from opentrons import simulate as OT2 # This line simulates the protocol
# Get the correct api version
protocol = OT2.get_protocol_api('2.10')
# Home the pipetting head
protocol.home()
# Call the 'run' function to run the protocol
run(protocol)
for line in protocol.commands():
    print(line)

C:\Users\bradl\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\bradl\.opentrons\deck_calibration.json not found. Loading defaults


Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 1
Transferring 100.0 from A2 of 3dprinted 15 Tube Rack 15000 ÂµL on 3 to A2 of Nunclon Delta Surface 163320 96 Well Plate 250 ÂµL on 4
Aspirating 100.0 uL from A2 of 3dprinted 15 Tube Rack 15000 ÂµL on 3 at 92.86 uL/sec
Dispensing 100.0 uL into A2 of Nunclon Delta Surface 163320 96 Well Plate 250 ÂµL on 4 at 92.86 uL/sec
Transferring 100.0 from A2 of 3dprinted 15 Tube Rack 15000 ÂµL on 3 to A3 of Nunclon Delta Surface 163320 96 Well Plate 250 ÂµL on 4
Aspirating 100.0 uL from A2 of 3dprinted 15 Tube Rack 15000 ÂµL on 3 at 92.86 uL/sec
Dispensing 100.0 uL into A3 of Nunclon Delta Surface 163320 96 Well Plate 250 ÂµL on 4 at 92.86 uL/sec
Transferring 100.0 from A2 of 3dprinted 15 Tube Rack 15000 ÂµL on 3 to A4 of Nunclon Delta Surface 163320 96 Well Plate 250 ÂµL on 4
Aspirating 100.0 uL from A2 of 3dprinted 15 Tube Rack 15000 ÂµL on 3 at 92.86 uL/sec
Dispensing 100.0 uL into A4 of Nunclon Delta Surface 163320 96 Well Plate 250 Âµ